This notebook is for...

Using visium data from [Garcia-Alsono et.al](https://www.ebi.ac.uk/arrayexpress/experiments/E-MTAB-9260/files/)
Overview also [here](https://docs.google.com/spreadsheets/d/1c7K7C5ZKNTSnSxgTO7oJte0juwFMqmX3sjeS4wjXy4I/edit#gid=1885672978)

Starting with the sample 152811, A13, early secretory phase.

Following the [scanpy-visium tutorial](https://scanpy-tutorials.readthedocs.io/en/latest/spatial/basic-analysis.html).

In [ ]:
import scanpy as sc
import squidpy as sq

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

In [ ]:
sc.logging.print_versions()
sc.set_figure_params(facecolor="white", figsize=(8, 8))
sc.settings.verbosity = 3

## Reading the data

In [ ]:
datadir = "data/GarciaAlsono_uterus/visium_data/uterus_152811/152811" # workstation

imageloc = "data/GarciaAlsono_uterus/visium_data/152811_20x_highest_res_image.jpg"

In [ ]:
adata = sc.read_visium(datadir,
                       count_file='152811_manual_filtered_feature_bc_matrix.h5',
                      # source_image_path='/data/local/rajewsky/home/vschuma/Jackie/data/GarciaAlsono_uterus/visium_data/')
                       source_image_path='data/GarciaAlsono_uterus/visium_data/') # workstation

In [ ]:
adata.var["feature_types"]

In [ ]:
adata.var_names_make_unique()
# adata.var['mt'] = adata.var_names.str.startswith("MT-")
# sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)

In [ ]:
# skipping the QC and preprocessing step bc. I think this stuff should be filtered/preprocessed already

## Manifold embedding and clustering based on transcriptional similarity

In [ ]:
# sc.pp.pca(adata)
# sc.pp.neighbors(adata)
# sc.tl.umap(adata)
# sc.tl.leiden(adata, key_added="clusters")

We plot some covariates to check if there is any particular structure in the UMAP associated with total counts and detected genes.

In [ ]:
# plt.rcParams["figure.figsize"] = (4, 4)
# sc.pl.umap(adata, color=["total_counts", "n_genes_by_counts", "clusters"], wspace=0.4)

## Visualization in spatial coordinates

In [ ]:
plt.rcParams["figure.figsize"] = (8, 8)
sc.pl.spatial(adata, img_key="hires", color=["total_counts", "n_genes_by_counts"])

 ## Cluster/show marker genes

In [ ]:
marker_genes = ['LGR5', # epithel
            'PAX2',# lumenal
            'SCGB2A2','PAEP', # glandular
            'IGF1', # stroma
            'CFD','CEBPB', 'PDGFRA', # secretory stroma
            'ACTA2', 'OGN' # fibroblasts
             ] # add marker genes from other notebook here
# make a loop to show the image for all the marker genes
sc.pl.spatial(adata, img_key="hires", color=marker_genes)

## Extract locations.txt and dge.txt from the anndata object

In [ ]:
# normalize expression data
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
adata

In [ ]:
adata.obsm['spatial'] # probably coordinates according to https://github.com/scverse/squidpy_notebooks/blob/main/docs/source/auto_tutorials/tutorial_read_spatial.rst

## Save atlas coordinates

In [ ]:
coord = adata.obsm['spatial']
datadir = "data/GarciaAlsono_uterus"
pd.DataFrame(data=coord, columns=['x','y']).to_csv(os.path.join(datadir,'endometr_coordinates.csv'))

## Creating the atlas and test it's uniqueness
atlas based on top 100 highly variable genes

This code is taken from the "data_for_celltpye_transfer_testing" notebook. I think it could be an intersting idea, to put this into a fucntion in a small script and add it to the repo.

In [ ]:
savepath = "data/GarciaAlsono_uterus/uterus_GarciaAlsonso_atlas_dge.txt"

# all highly variable genes from which I will subsample
high_var_index = adata.var[adata.var['highly_variable'] == True].index
high_var_index

# subsampling
marker_subsample_names = high_var_index.to_frame().sample(150)
# marker_subsample_names = adata.var_names.to_frame().sample(50)
marker_subsample_idx = []
for gene in marker_subsample_names[0] :
    marker_subsample_idx.append(adata.var_names.get_loc(gene))
marker_subsample = pd.DataFrame(adata.X[:, marker_subsample_idx].toarray(),
                                columns=marker_subsample_names[0])
# count sets
loc_w_gene = 0
loc_wo_gene = 0
for idx,row in marker_subsample.iterrows():
    if (row == 0).all():
        loc_wo_gene += 1
    else:
        loc_w_gene += 1
marker_cov = loc_w_gene/marker_subsample.shape[0]
print('num loc w gene = {} \n'
      'num loc w/o gene = {}\n'
      'therefore gene marker cov = {}'.format(loc_w_gene, loc_wo_gene, marker_cov))

# evaluate point uniqueness
# 1. binarize marker_subsample
marker_subsample_bin = marker_subsample.mask(marker_subsample > 0, 1 )
# 2. vector of gene sets
gene_sets_p_loc = []
# 3. get gene set per location
for idx, row in marker_subsample_bin.iterrows():
    gene_sets_p_loc.append(
        tuple(row.index[row.loc[:] == 1])
    )
# 4. count how many unique sets are in the list
num_unique = len(set(gene_sets_p_loc))

# 5. calculate ratio of covered locations that are unique
unique_cov = num_unique/(marker_subsample.shape[0] - loc_wo_gene)
print("{:.2%} of the marker-locations are unique.".format(unique_cov))

# 6. write atlas to file
if marker_cov == 1.0 and unique_cov == 1.0:
    marker_subsample.to_csv(savepath, index_label=False)
else:
    print("you should resample before saving this")

## getting spatialy informative genes
following [this squidpy tutorial](https://squidpy.readthedocs.io/en/stable/auto_tutorials/tutorial_visium_hne.html#spatially-variable-genes-with-moran-s-i)

In [ ]:
spatial_info_genes = adata[:, adata.var.highly_variable].var_names.values[:1000]
sq.gr.spatial_neighbors(adata)
sq.gr.spatial_autocorr(
    adata,
    mode="moran",
    genes=spatial_info_genes,
    n_perms=100,
    n_jobs=1,
)

### make atlas from spatially informative genes

In [ ]:
savepath = "data/GarciaAlsono_uterus/uterus_GarciaAlsonso_atlas_spatial_info_dge.csv"

# all spatially informative genes from which I will subsample
spat_info_index = adata.uns["moranI"].head(200).index
spat_info_index

# subsampling
marker_subsample_names = spat_info_index.to_frame().sample(80)
# marker_subsample_names = adata.var_names.to_frame().sample(50)
marker_subsample_idx = []
for gene in marker_subsample_names[0] :
    marker_subsample_idx.append(adata.var_names.get_loc(gene))
marker_subsample = pd.DataFrame(adata.X[:, marker_subsample_idx].toarray(),
                                columns=marker_subsample_names[0])
# count sets
loc_w_gene = 0
loc_wo_gene = 0
for idx,row in marker_subsample.iterrows():
    if (row == 0).all():
        loc_wo_gene += 1
    else:
        loc_w_gene += 1
marker_cov = loc_w_gene/marker_subsample.shape[0]
print('num loc w gene = {} \n'
      'num loc w/o gene = {}\n'
      'therefore gene marker cov = {}'.format(loc_w_gene, loc_wo_gene, marker_cov))

# evaluate point uniqueness
# 1. binarize marker_subsample
marker_subsample_bin = marker_subsample.mask(marker_subsample > 0, 1 )
# 2. vector of gene sets
gene_sets_p_loc = []
# 3. get gene set per location
for idx, row in marker_subsample_bin.iterrows():
    gene_sets_p_loc.append(
        tuple(row.index[row.loc[:] == 1])
    )
# 4. count how many unique sets are in the list
num_unique = len(set(gene_sets_p_loc))

# 5. calculate ratio of covered locations that are unique
unique_cov = num_unique/(marker_subsample.shape[0] - loc_wo_gene)
print("{:.2%} of the marker-locations are unique.".format(unique_cov))

# 6. write atlas to file
if marker_cov == 1.0 and unique_cov == 1.0:
    marker_subsample.to_csv(savepath, index_label=False)
else:
    print("you should resample before saving this")

### make atlas from spatially informative genes

In [ ]:
savepath = "data/GarciaAlsono_uterus/uterus_GarciaAlsonso_atlas_spatial_info_dge.csv"

# all spatially informative genes from which I will subsample
spat_info_index = adata.uns["moranI"].head(200).index
spat_info_index

# subsampling
marker_subsample_names = spat_info_index.to_frame().sample(80)
# marker_subsample_names = adata.var_names.to_frame().sample(50)
marker_subsample_idx = []
for gene in marker_subsample_names[0] :
    marker_subsample_idx.append(adata.var_names.get_loc(gene))
marker_subsample = pd.DataFrame(adata.X[:, marker_subsample_idx].toarray(),
                                columns=marker_subsample_names[0])
# count sets
loc_w_gene = 0
loc_wo_gene = 0
for idx,row in marker_subsample.iterrows():
    if (row == 0).all():
        loc_wo_gene += 1
    else:
        loc_w_gene += 1
marker_cov = loc_w_gene/marker_subsample.shape[0]
print('num loc w gene = {} \n'
      'num loc w/o gene = {}\n'
      'therefore gene marker cov = {}'.format(loc_w_gene, loc_wo_gene, marker_cov))

# evaluate point uniqueness
# 1. binarize marker_subsample
marker_subsample_bin = marker_subsample.mask(marker_subsample > 0, 1 )
# 2. vector of gene sets
gene_sets_p_loc = []
# 3. get gene set per location
for idx, row in marker_subsample_bin.iterrows():
    gene_sets_p_loc.append(
        tuple(row.index[row.loc[:] == 1])
    )
# 4. count how many unique sets are in the list
num_unique = len(set(gene_sets_p_loc))

# 5. calculate ratio of covered locations that are unique
unique_cov = num_unique/(marker_subsample.shape[0] - loc_wo_gene)
print("{:.2%} of the marker-locations are unique.".format(unique_cov))

# 6. write atlas to file
if marker_cov == 1.0 and unique_cov == 1.0:
    marker_subsample.to_csv(savepath, index_label=False)
else:
    print("you should resample before saving this")

In [29]:
savepath = "data/GarciaAlsono_uterus/uterus_GarciaAlsonso_atlas_spatial_info_dge.csv"

# all spatially informative genes from which I will subsample
spat_info_index = adata.uns["moranI"].head(200).index
spat_info_index

# subsampling
marker_subsample_names = spat_info_index.to_frame().sample(80)
# marker_subsample_names = adata.var_names.to_frame().sample(50)
marker_subsample_idx = []
for gene in marker_subsample_names[0] :
    marker_subsample_idx.append(adata.var_names.get_loc(gene))
marker_subsample = pd.DataFrame(adata.X[:, marker_subsample_idx].toarray(),
                                columns=marker_subsample_names[0])
# count sets
loc_w_gene = 0
loc_wo_gene = 0
for idx,row in marker_subsample.iterrows():
    if (row == 0).all():
        loc_wo_gene += 1
    else:
        loc_w_gene += 1
marker_cov = loc_w_gene/marker_subsample.shape[0]
print('num loc w gene = {} \n'
      'num loc w/o gene = {}\n'
      'therefore gene marker cov = {}'.format(loc_w_gene, loc_wo_gene, marker_cov))

# evaluate point uniqueness
# 1. binarize marker_subsample
marker_subsample_bin = marker_subsample.mask(marker_subsample > 0, 1 )
# 2. vector of gene sets
gene_sets_p_loc = []
# 3. get gene set per location
for idx, row in marker_subsample_bin.iterrows():
    gene_sets_p_loc.append(
        tuple(row.index[row.loc[:] == 1])
    )
# 4. count how many unique sets are in the list
num_unique = len(set(gene_sets_p_loc))

# 5. calculate ratio of covered locations that are unique
unique_cov = num_unique/(marker_subsample.shape[0] - loc_wo_gene)
print("{:.2%} of the marker-locations are unique.".format(unique_cov))

# 6. write atlas to file
if marker_cov == 1.0 and unique_cov == 1.0:
    marker_subsample.to_csv(savepath, index_label=False)
else:
    print("you should resample before saving this")

num loc w gene = 3871 
num loc w/o gene = 0
therefore gene marker cov = 1.0
100.00% of the marker-locations are unique.
